In [1]:
import pandas as pd

In [2]:
#wczytanie plikow
zamowienia = pd.read_csv('zamowienia.csv', sep = ';')
wyslane = pd.read_csv('wyslane.csv', sep = ';')
cena_bazowa = pd.read_csv('cena bazowa.csv', sep = ';')

In [45]:
print(zamowienia.head(), '\n\n', zamowienia.dtypes)

   order_id  client_id  order_date KUPON
0   1462414  523933015  29.06.2019   NaN
1   1538139  814107372  15.08.2019   NaN
2   1838871  713139458  19.08.2019   NaN
3   1896544  123641536  05.05.2019   15%
4   1943852  602252913  30.07.2019   NaN 

 order_id       int64
client_id      int64
order_date    object
KUPON         object
dtype: object


In [10]:
print(wyslane.head())
print(wyslane.dtypes)

    order_id  item_id_1  item_id_2  item_id_3
0  314631735    2655227  9051607.0        NaN
1  802224752    1984910  4286301.0        NaN
2  716057036    2669985  1052038.0        NaN
3  125731133    5966397        NaN        NaN
4   12070864    6788106  9643817.0        NaN
order_id       int64
item_id_1      int64
item_id_2    float64
item_id_3    float64
dtype: object


In [11]:
print(cena_bazowa.head())
print(cena_bazowa.dtypes)

   item_id  base_price
0  7447126         225
1  2696692         112
2  2883136         192
3  1984910         136
4  2242676         107
item_id       int64
base_price    int64
dtype: object


# Zadanie 1
**Zad.1.1** W jakim dniu roku klienci złożyli najwięcej zamówień?

In [13]:
#czy columna 'order_id' zawiera duplikaty?
zamowienia['order_id'].duplicated().any()


True

In [16]:
#pogrupuj tabelę ze wzgl. na 'order_date' i licz unikatowe 'order_id', sortuj majejąco
zamowienia.groupby('order_date').nunique().sort_values('order_id', ascending=False)['order_id']

order_date
14.08.2019    41
15.04.2019    39
06.05.2019    39
18.06.2019    39
19.03.2019    38
              ..
25.08.2019    18
21.06.2019    17
08.07.2019    16
09.03.2019    14
24.09.2019     1
Name: order_id, Length: 182, dtype: int64

,**ODP**: Najwięcej zamówień zrealizowano 14stego sierpnia 2019 roku

**Zad.1.2**  Ilu klientów skorzystało z kuponu rabatowego w trakcie zakupów?

In [22]:
#W celach kontrolnych sprawdź ile jest klientow w bazie
l_klientow = zamowienia['client_id'].nunique()
print(l_klientow)

5000


In [33]:
#Stwórz tabelę z rekordami niezawierającymi brakującej wartości w 'KUPON'
klient_kupon = zamowienia.loc[~zamowienia['KUPON'].isna()]
print(klient_kupon.head(20), '\n')
print(klient_kupon.tail(10))

     order_id  client_id  order_date KUPON
3     1896544  123641536  05.05.2019   15%
5     2143538  563294140  08.04.2019   40%
23    4932443  314452019  31.03.2019   20%
25    5231560  649526055  19.07.2019    5%
26    6074529  849773609  08.07.2019   30%
34    7643295  380338614  01.07.2019   20%
37    7760735  432992730  25.07.2019   33%
43    8491185  560082186  06.06.2019   22%
44    8498697  782072858  03.04.2019   47%
87   17060830   38976820  06.06.2019   43%
101  19081937  653782027  16.06.2019   11%
120  22511692  906035383  18.04.2019   37%
122  23523052  903152127  24.07.2019   13%
124  23786262  633370291  26.08.2019   38%
127  24976074   72002508  23.05.2019   22%
128  25549564  214128780  07.07.2019   20%
142  26633023  826087161  22.05.2019   27%
143  26831396  105444091  21.06.2019    7%
145  27079353  807481688  10.07.2019   29%
146  28079090  832378906  23.07.2019   33% 

       order_id  client_id  order_date KUPON
4978  989975805  629069382  11.03.2019   23%
4983 

In [32]:
#Zlicz liczbę unikatowych wartości 'client_id' w tej tabeli
odp = len(klient_kupon['client_id'].unique())
print(odp)

998


**ODP**: Zakupy z rabatem wykonało 998 klientów.

**Zad1.3** Ilu klientów zrobiło w analizowanym okresie więcej niż jedno zamówienie?

In [38]:
#Zlicz wartości 'client_id', dla których występuje > 1 niepowtarzająca się wartość 'order_id'
zam = zamowienia.groupby('client_id').nunique()
zam.head()

,order_id,client_id,order_date,KUPON
client_id,,,,
107893,1,1,1,0
165795,1,1,1,1
288561,1,1,1,0
554772,1,1,1,1
713722,1,1,1,0


In [40]:
odp = len(zam.loc[zam['order_id'] > 1])
print(odp)

17


**ODP:** Więcej niż 1 zamówienie wykonało 17 klientów.

## **Zad.1.4** Który z produktów cieszył się największym powodzeniem? Ilu klientów kupiło go ze zniżką?

In [47]:
wyslane['order_id'].duplicated().any()

False

In [53]:
#Zacznę od stworzenia tabeli (na podstawie 'wyslane') gdzie wartość każdego 'produkt_id' będzie w osobnym wierszu
# i pogrupowania jej ze względu na wartość 'product_id'
wyslane_order_item1 = pd.DataFrame()
wyslane_order_item2 = pd.DataFrame()
wyslane_order_item3 = pd.DataFrame()
wyslane_order_item1[['order_id', 'item_id']] = wyslane[['order_id','item_id_1']]
wyslane_order_item2[['order_id', 'item_id']] = wyslane[['order_id','item_id_2']]
wyslane_order_item3[['order_id', 'item_id']] = wyslane[['order_id','item_id_3']]
wyslane_new = pd.concat([wyslane_order_item1,wyslane_order_item2, wyslane_order_item3])
wyslane_new.groupby(['item_id']).count().sort_values(by='order_id', ascending=False)['order_id']


item_id
4355722.0    70
4652599.0    65
8368010.0    64
4550408.0    64
4118332.0    62
             ..
6837984.0    37
534353.0     37
1816982.0    35
4666241.0    33
7250913.0    31
Name: order_id, Length: 110, dtype: int64

In [57]:
#Na tym etapie wiemy, że najczęściej wysyłany produkt to produkt o 'item_id' = 4355722.
#Sprawdźmy ilu klientów kupiło ten produkt ze zniżką.
print(wyslane_new.sort_values('order_id').head(), '\n\n' , len(wyslane_new))


      order_id    item_id
1295   1462414        NaN
1295   1462414  1115180.0
1295   1462414        NaN
2744   1538139  7838989.0
2744   1538139  3109031.0 

 11001


In [63]:
#usuń wiersze, w których wartość 'item_id' jest brakująca.
wys_clean = wyslane_new.loc[~wyslane_new['item_id'].isna()]
wys_clean.sort_values('order_id').head(15)
wys_produkt = wyslane_new.loc[wyslane_new['item_id'] == 4355722]
wys_produkt

,order_id,item_id
7,533840397,4355722.0
97,8769369,4355722.0
288,686785833,4355722.0
367,935081843,4355722.0
410,626848390,4355722.0
...,...,...
3357,250983304,4355722.0
3361,149348383,4355722.0
3490,974122825,4355722.0
3513,145853928,4355722.0


In [65]:
#połącz tabelę wys_produkt z tabelą wyslane lewostronnie poprzez kolumnę 'order_id'
zamow_produkt = pd.merge(left = wys_produkt, right = zamowienia, on = 'order_id', how = 'left')
len(zamow_produkt) == len(wys_produkt)

True

In [66]:
zamow_produkt.head(10)

,order_id,item_id,client_id,order_date,KUPON
0,533840397,4355722.0,1394242,04.06.2019,NaN
1,8769369,4355722.0,292592441,06.03.2019,NaN
2,686785833,4355722.0,327247246,23.08.2019,NaN
3,935081843,4355722.0,340242212,10.04.2019,NaN
4,626848390,4355722.0,349194620,09.07.2019,NaN
5,719996443,4355722.0,367562101,15.04.2019,NaN
6,915988214,4355722.0,370886321,17.05.2019,NaN
7,224480672,4355722.0,413811797,13.06.2019,NaN
8,237748346,4355722.0,438622704,28.05.2019,NaN
9,138971790,4355722.0,464167862,19.06.2019,27%


In [72]:
prd_kupon = zamow_produkt.loc[~zamow_produkt['KUPON'].isna()]
prd_kupon.head()

,order_id,item_id,client_id,order_date,KUPON
9,138971790,4355722.0,464167862,19.06.2019,27%
21,314897786,4355722.0,648064241,05.05.2019,25%
22,462374358,4355722.0,671157869,24.04.2019,24%
31,383098201,4355722.0,884329427,11.08.2019,45%
32,877092857,4355722.0,895450181,06.03.2019,25%


In [74]:
l_klientow = prd_kupon['client_id'].nunique()
print(l_klientow)

11


**ODP:** Najpopularniejszy produkt to 4355722, a 11 klientów kupiło go ze zniżką.

## **Zad.1.5** Który produkt był najczęściej kupowany ze zniżką?

In [77]:
prod_znizka = pd.merge(left = wys_clean, right = zamowienia, on = 'order_id', how = 'left')
prod_znizka.head()

,order_id,item_id,client_id,order_date,KUPON
0,314631735,2655227.0,107893,19.05.2019,NaN
1,802224752,1984910.0,165795,26.03.2019,49%
2,716057036,2669985.0,288561,06.03.2019,NaN
3,125731133,5966397.0,554772,04.04.2019,5%
4,12070864,6788106.0,713722,04.03.2019,NaN


In [79]:
prod_znizka.loc[~prod_znizka['KUPON'].isna()].groupby('item_id').count().sort_values('KUPON', ascending=False)

,order_id,client_id,order_date,KUPON
item_id,,,,
1554376.0,19,19,19,19
9873648.0,17,17,17,17
552870.0,16,16,16,16
5007446.0,15,15,15,15
7642810.0,15,15,15,15
...,...,...,...,...
4666241.0,5,5,5,5
1816982.0,4,4,4,4
1115421.0,4,4,4,4


**ODP:** Ze zniżką najczęściej kupowano produkt 1554376. 
## Zad.1.6 Jaka była końcowa wartość wszystkich zamówień w badanym okresie?

In [85]:
# z tabeli 'prod_znizka' wyodrębnić kolumny 'item_id' oraz 'KUPON' i zamienić dtype na float
znizka = prod_znizka[['item_id', 'KUPON']]
znizka['wartosc_znizki'] = znizka['KUPON'].str.rstrip('%').astype('float')/100
znizka.head(15)

/home/marcelina/Documents/Python/1-envir/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,item_id,KUPON,wartosc_znizki
0,2655227.0,NaN,NaN
1,1984910.0,49%,0.49
2,2669985.0,NaN,NaN
3,5966397.0,5%,0.05
4,6788106.0,NaN,NaN
5,4652599.0,NaN,NaN
6,2940993.0,NaN,NaN
7,4355722.0,NaN,NaN
8,9038883.0,7%,0.07
9,5168890.0,NaN,NaN


In [87]:
#połaczyc tabele znizka z cena bazowa (pracując z realnymi danymi 
#zastanowiłoby mnie, dlaczego cena bazowa za każdym razem jest liczbą całkowitą)

do_sumowania = pd.merge(left = znizka, right = cena_bazowa, on='item_id', how = 'left')
do_sumowania.head(10)


,item_id,KUPON,wartosc_znizki,base_price
0,2655227.0,NaN,NaN,122
1,1984910.0,49%,0.49,136
2,2669985.0,NaN,NaN,142
3,5966397.0,5%,0.05,226
4,6788106.0,NaN,NaN,117
5,4652599.0,NaN,NaN,152
6,2940993.0,NaN,NaN,232
7,4355722.0,NaN,NaN,178
8,9038883.0,7%,0.07,155
9,5168890.0,NaN,NaN,230


In [90]:
do_sumowania.loc[do_sumowania['wartosc_znizki'].isna(), 'wartosc_znizki'] = 0
do_sumowania['cena_wiersza'] = do_sumowania['base_price'] - (do_sumowania['wartosc_znizki'] * do_sumowania['wartosc_znizki'])

do_sumowania.head(15)

,item_id,KUPON,wartosc_znizki,base_price,cena_wiersza
0,2655227.0,NaN,0.00,122,122.0000
1,1984910.0,49%,0.49,136,135.7599
2,2669985.0,NaN,0.00,142,142.0000
3,5966397.0,5%,0.05,226,225.9975
4,6788106.0,NaN,0.00,117,117.0000
5,4652599.0,NaN,0.00,152,152.0000
6,2940993.0,NaN,0.00,232,232.0000
7,4355722.0,NaN,0.00,178,178.0000
8,9038883.0,7%,0.07,155,154.9951
9,5168890.0,NaN,0.00,230,230.0000


In [93]:
wyniki = do_sumowania['cena_wiersza'].sum()
print(wyniki)

839016.8334


**ODP:** Łączna kwota wszystkich zamówień (z uwzględnieniem zniżek) wyniosła 839016,8334 (',' to znak dziesiętny)